In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keplergl import KeplerGl
from datetime import datetime
from shapely.geometry import LineString, shape, Point
from shapely import to_geojson
import zipfile
import os
import osmnx as ox
from geopy.geocoders import Nominatim
from typing import List
import h3
import ast
import altair as alt
from shapely import Point

%matplotlib inline

In [3]:
#geom is ecpected to be geojson format
### take care to take that one level of the nested json --> ['features'][0]

def shape_filter(df, geom, bbox:bool=False):
    lng_min, lng_max, lat_min, lat_max = 100, 0, 100, 0
    geom_shape = geom['geometry']['coordinates'][0]
    for i in geom_shape:
        if i[0] < lng_min:
            lng_min = i[0]
        elif i[0] > lng_max:
            lng_max = i[0]
        else:
            pass
        if i[1] < lat_min:
            lat_min = i[1]
        elif i[1] > lat_max:
            lat_max = i[1]
        else:
            pass

    sub_df = df[df['Longitude'] > lng_min]
    sub_df = sub_df[sub_df['Longitude'] < lng_max]
    sub_df = sub_df[sub_df['Latitude'] > lat_min]
    sub_df = sub_df[sub_df['Latitude'] < lat_max]

    if bbox==False:
        sub_df['geo_check'] = sub_df.apply(lambda x: polygon_check(x['Longitude'], x['Latitude'],geom), axis=1)
        sub_df = sub_df[sub_df['geo_check']].drop('geo_check',axis=1)
    sub_df.reset_index(drop=True,inplace=True)
    return sub_df

In [4]:
df = pd.read_csv('../data/interim/waypoints.csv', header=None)

In [5]:
header = pd.read_csv('../data/interim/TripBulkReportWaypointsHeaders.csv')

In [6]:
df.drop([11], axis=1, inplace=True)

In [8]:
df.columns = header.columns

### filter trip starts in district 1

In [9]:
district1 = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              14.3997219836956,
              50.08946066514625
            ],
            [
              14.3997219836956,
              50.08568425669637
            ],
            [
              14.409012189651008,
              50.08568425669637
            ],
            [
              14.409012189651008,
              50.08946066514625
            ],
            [
              14.3997219836956,
              50.08946066514625
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [10]:
trip_df = pd.read_csv('../data/interim/trips.csv',header=None,on_bad_lines='skip')
headers = pd.read_csv('../data/interim/TripBulkReportTripsHeaders.csv')
trip_df.drop([31], axis=1, inplace=True)
trip_df.columns = headers.columns

In [12]:
trip_df['Longitude'] = trip_df['StartLocLon']
trip_df['Latitude'] = trip_df['StartLocLat']

In [13]:
trip_df.head()

,TripId,DeviceId,ProviderId,Mode,StartDate,StartWDay,EndDate,EndWDay,StartLocLat,StartLocLon,...,MovementType,OriginCbg,DestCbg,StartTimezone,EndTimezone,WaypointFreqSec,StartQk,EndQk,Longitude,Latitude
0,00680db75571fdb2ed00fc87b1b419ef,8867e80fcdbb3d6b74e73339f72ef8dc,98b297950041a42470269d56260243a1,1,2024-08-06T13:30:57.000Z,2,2024-08-06T14:03:07.000Z,2,50.0753,14.4336,...,1,NaN,NaN,Europe/Prague,Europe/Prague,10.052083,120212302122221132,120212302122221332,14.4336,50.0753
1,00e7ac3b46277d46b9cc31d4821cfea4,69c400ce2c69ad168e0366f65244bb41,7fe1f8abaad094e0b5cb1b01d712f708,1,2024-08-07T11:07:37.000Z,3,2024-08-07T13:42:37.000Z,3,50.0654,14.5404,...,1,NaN,NaN,Europe/Prague,Europe/Warsaw,290.625000,120212302123233320,120212311202310133,14.5404,50.0654
2,0135963269dbb22d3bb82ddd46b25d46,c83fad98fe1936a88a7d85135e91b8f6,8e6b42f1644ecb1327dc03ab345e618b,1,2024-08-04T09:32:56.999Z,7,2024-08-04T10:37:18.000Z,7,49.5178,15.3967,...,1,NaN,NaN,Europe/Prague,Europe/Prague,5.182552,120212321313203011,120212302021102112,15.3967,49.5178
3,0289a5664287f93a28157cfe95df3042,e14ac160a67a08a25c7422f16b561ec2,7fe1f8abaad094e0b5cb1b01d712f708,1,2024-08-06T10:32:49.000Z,2,2024-08-06T11:04:06.000Z,2,50.0932,14.3988,...,1,NaN,NaN,Europe/Prague,Europe/Prague,11.658385,120212302033132322,120212302120330233,14.3988,50.0932
4,045c25562a5f7fcc906b08f4175b2dad,8d4540b29323e3bfba4db57be982b5f5,f4f6dce2f3a0f9dada0c2b5b66452017,1,2024-08-07T09:54:51.000Z,3,2024-08-07T10:00:40.000Z,3,50.0351,14.3060,...,1,NaN,NaN,Europe/Prague,Europe/Prague,49.857143,120212302210310001,120212302210322033,14.3060,50.0351


In [14]:
gdf = gpd.GeoDataFrame.from_features(district1)
gdf

,geometry
0,"POLYGON ((14.39972 50.08946, 14.39972 50.08568..."


In [16]:
df['geometry'] = df.apply(lambda x: Point(x['Longitude'], x['Latitude']), axis=1)

In [17]:
gdf_wp = gpd.GeoDataFrame(df, geometry=df['geometry'])

In [18]:
gdf_within = gpd.sjoin(gdf_wp, gdf, how="inner", op="within")

/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [19]:
gdf_within.head()

,TripId,WaypointSequence,CaptureDate,Latitude,Longitude,SegmentId,ZoneName,Frc,DeviceId,RawSpeed,RawSpeedMetric,geometry,index_right
54631,abe3c2039c444e7b10ecafd0236e5a17,92,2024-08-05T10:29:10.000Z,50.089329,14.399761,NaN,NaN,NaN,b7b7e6f1dc9286630209fceafd6dff99,NaN,kph,POINT (14.39976 50.08933),0
54632,abe3c2039c444e7b10ecafd0236e5a17,93,2024-08-05T10:29:13.000Z,50.089307,14.399822,NaN,NaN,NaN,b7b7e6f1dc9286630209fceafd6dff99,NaN,kph,POINT (14.39982 50.08931),0
54633,abe3c2039c444e7b10ecafd0236e5a17,94,2024-08-05T10:29:16.000Z,50.089308,14.399869,NaN,NaN,NaN,b7b7e6f1dc9286630209fceafd6dff99,NaN,kph,POINT (14.39987 50.08931),0
54634,abe3c2039c444e7b10ecafd0236e5a17,95,2024-08-05T10:29:19.000Z,50.089310,14.399888,NaN,NaN,NaN,b7b7e6f1dc9286630209fceafd6dff99,NaN,kph,POINT (14.39989 50.08931),0
54635,abe3c2039c444e7b10ecafd0236e5a17,96,2024-08-05T10:29:22.000Z,50.089311,14.399920,NaN,NaN,NaN,b7b7e6f1dc9286630209fceafd6dff99,NaN,kph,POINT (14.39992 50.08931),0


### merge with waypoints

In [126]:
district_data = pd.merge(df, gdf_within, how='inner', on='TripId')

: 

In [119]:
district_data.columns

Index(['TripId', 'WaypointSequence_x', 'CaptureDate_x', 'Latitude_x',
       'Longitude_x', 'SegmentId_x', 'ZoneName_x', 'Frc_x', 'DeviceId_x',
       'RawSpeed_x', 'RawSpeedMetric_x', 'geometry_x', 'WaypointSequence_y',
       'CaptureDate_y', 'Latitude_y', 'Longitude_y', 'SegmentId_y',
       'ZoneName_y', 'Frc_y', 'DeviceId_y', 'RawSpeed_y', 'RawSpeedMetric_y',
       'geometry_y', 'index_right'],
      dtype='object')

In [122]:
cols = ['TripId', 'CaptureDate_x', 'RawSpeedMetric_x', 'RawSpeed_x', 'geometry_x']

In [1]:
district_data = district_data[cols]

NameError: name 'district_data' is not defined

In [125]:
district_data

['TripId', 'CaptureDate_x', 'RawSpeedMetric_x', 'RawSpeed_x', 'geometry_x']

In [118]:
test = district_data[0:10000]

In [ ]:
test 